# Initiate arrays

In [ ]:
stage_array = ['13', '14', '15', '17', '18', '19', '25']
datatype_array = ['occ', 'nucleoatac_signal', 'nucleoatac_signal.smooth']

In [ ]:
for stage in stage_array:
    stagename = stage + 'AB_nucleoatac'
    !echo '### Now analyzing sample '$stagename
    
    for datatype in datatype_array:
        filename = 'S' + stagename + '.' + datatype + '.bedgraph.gz'
        gunzipped = filename.replace('.gz', '')
        !echo '### gunzipping '$filename' into '$gunzipped
        !gunzip -c $filename > $gunzipped
        
        bwname = gunzipped.replace('.bedgraph', '.bw')
        !echo '### converting '$gunzipped' into '$bwname
        !./bedGraphToBigWig $gunzipped phaw_5.0.chrom.sizes $bwname
        
        refpoint_name = bwname.replace('.bw', '.refpoint.gz')
        !echo '### computing refpoint matrix for '$bwname
        !computeMatrix reference-point -S $bwname --regionsFileName phaw_5.0.eggnog.sorted.mRNA.bed --outFileName $refpoint_name -b 400 -a 800 --numberOfProcessors max --skipZeros --missingDataAsZero
        
        refpoint_pdf = refpoint_name.replace('.gz', '.pdf')
        !echo '### drawing figure into '$refpoint_pdf
        !plotHeatmap --matrixFile $refpoint_name --outFileName $refpoint_pdf
        
        scaled_name = bwname.replace('.bw', '.scaled.gz')
        !echo '### computing scaled-regions matrix for '$bwname
        !computeMatrix scale-regions -S $bwname --regionsFileName phaw_5.0.eggnog.sorted.mRNA.bed --outFileName $scaled_name -b 400 -a 400 --numberOfProcessors max --skipZeros --missingDataAsZero
        
        scaled_pdf = scaled_name.replace('.gz', '.pdf')
        !echo '### drawing figure into '$scaled_pdf
        !plotHeatmap --matrixFile $scaled_name --outFileName $scaled_pdf
        
    !echo '### done analyzing '$stagename

!mv *.scaled.pdf NucATAC_pdfs
!mv *.refpoint.pdf NucATAC_pdfs

In [ ]:
datagroup_array = ['S*AB_nucleoatac.nucleoatac_signal.bw', 'S*AB_nucleoatac.nucleoatac_signal.smooth.bw', 'S*AB_nucleoatac.occ.bw']

for datagroup in datagroup_array:
    output = datagroup.replace('*', 'all').replace('.bw', '.refpoint.gz')
    !echo '### computing refpoint matrix for '$datagroup
    !computeMatrix reference-point -S $datagroup --regionsFileName phaw_5.0.eggnog.sorted.mRNA.bed --outFileName $output -b 400 -a 800 --numberOfProcessors max --skipZeros --missingDataAsZero
    
    output_pdf = output.replace('.gz', '.pdf')
    !echo '### drawing figure into '$output_pdf
    !plotHeatmap --matrixFile $output --outFileName $output_pdf